In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
display_table('Activities')

+------------+------------+
| sell_date  |  product   |
+------------+------------+
| 2020-05-30 | Headphone  |
| 2020-06-01 |   Pencil   |
| 2020-06-02 |    Mask    |
| 2020-05-30 | Basketball |
| 2020-06-01 |   Bible    |
| 2020-06-02 |    Mask    |
| 2020-05-30 |  T-Shirt   |
+------------+------------+


### Find for each date the number of different products sold and their names.The sold products names for each date should be sorted lexicographically. Return the result table ordered by sell_date.

```
+------------+----------+------------------------------+
| sell_date  | num_sold | products                     |
+------------+----------+------------------------------+
| 2020-05-30 | 3        | Basketball,Headphone,T-shirt |
| 2020-06-01 | 2        | Bible,Pencil                 |
| 2020-06-02 | 1        | Mask                         |
+------------+----------+------------------------------+
Explanation: 
For 2020-05-30, Sold items were (Headphone, Basketball, T-shirt), we sort them lexicographically and separate them by a comma.
For 2020-06-01, Sold items were (Pencil, Bible), we sort them lexicographically and separate them by a comma.
For 2020-06-02, the Sold item is (Mask), we just return it.
```

In [3]:
%%sql 

SELECT 
    sell_date, 
    COUNT(DISTINCT product) AS num_sold
FROM Activities
GROUP BY sell_date
ORDER BY sell_date;

sell_date,num_sold
2020-05-30,3
2020-06-01,2
2020-06-02,1


In [4]:
%%sql 

SELECT 
    sell_date, 
    COUNT(DISTINCT product) AS num_sold, 
    GROUP_CONCAT(product) AS products
FROM Activities
GROUP BY sell_date
ORDER BY sell_date;

sell_date,num_sold,products
2020-05-30,3,"Headphone,Basketball,T-Shirt"
2020-06-01,2,"Pencil,Bible"
2020-06-02,1,"Mask,Mask"


In [5]:
%%sql 

SELECT 
    sell_date, 
    COUNT(DISTINCT product) AS num_sold, 
    GROUP_CONCAT(DISTINCT product) AS products
FROM Activities
GROUP BY sell_date
ORDER BY sell_date;

sell_date,num_sold,products
2020-05-30,3,"Basketball,Headphone,T-Shirt"
2020-06-01,2,"Bible,Pencil"
2020-06-02,1,Mask


# Using Pandas

In [6]:
import pandas as pd 

In [7]:
activities_query = %sql SELECT * FROM Activities # type: ignore
activities_df = activities_query.DataFrame()

display(activities_df)

,sell_date,product
0,2020-05-30,Headphone
1,2020-06-01,Pencil
2,2020-06-02,Mask
3,2020-05-30,Basketball
4,2020-06-01,Bible
5,2020-06-02,Mask
6,2020-05-30,T-Shirt


In [8]:
activities_df = activities_df.drop_duplicates()
activities_df

,sell_date,product
0,2020-05-30,Headphone
1,2020-06-01,Pencil
2,2020-06-02,Mask
3,2020-05-30,Basketball
4,2020-06-01,Bible
6,2020-05-30,T-Shirt


In [9]:
activities_df.groupby('sell_date')['product'].agg(''.join)

sell_date
2020-05-30    HeadphoneBasketballT-Shirt
2020-06-01                   PencilBible
2020-06-02                          Mask
Name: product, dtype: object

In [10]:
activities_df.groupby('sell_date')['product'].agg(','.join)

sell_date
2020-05-30    Headphone,Basketball,T-Shirt
2020-06-01                    Pencil,Bible
2020-06-02                            Mask
Name: product, dtype: object

In [11]:
activities_df.groupby('sell_date')['product'].agg(','.join).reset_index()

,sell_date,product
0,2020-05-30,"Headphone,Basketball,T-Shirt"
1,2020-06-01,"Pencil,Bible"
2,2020-06-02,Mask


In [12]:
activities_df.groupby('sell_date')['product'].agg(lambda x: ','.join(x))

sell_date
2020-05-30    Headphone,Basketball,T-Shirt
2020-06-01                    Pencil,Bible
2020-06-02                            Mask
Name: product, dtype: object

In [13]:
grouped_df = activities_df.groupby('sell_date')['product'] \
            .agg(lambda x: ','.join(sorted(x))) \
            .reset_index()
grouped_df

,sell_date,product
0,2020-05-30,"Basketball,Headphone,T-Shirt"
1,2020-06-01,"Bible,Pencil"
2,2020-06-02,Mask


In [16]:
grouped_df.rename(columns={'product': 'products'}, inplace=True)
grouped_df

,sell_date,products
0,2020-05-30,"Basketball,Headphone,T-Shirt"
1,2020-06-01,"Bible,Pencil"
2,2020-06-02,Mask
